### Noticias 

In [1]:
import requests
import pandas as pd
import datetime as dt
from transformers import pipeline


In [2]:
# Configuración
NEWS_API_KEY = 'c6a6a47f2cfb4a3c8259b57247d1149c'  # Reemplaza con tu clave API
QUERY = 'Bitcoin'
FROM_DATE = (dt.datetime.now() - dt.timedelta(days=30)).strftime('%Y-%m-%d')  # Últimos 30 días
TO_DATE = dt.datetime.now().strftime('%Y-%m-%d')


In [3]:
def obtener_noticias(query, from_date, to_date, api_key, page=1):
    url = ('https://newsapi.org/v2/everything?'
           f'q={query}&'
           f'from={from_date}&'
           f'to={to_date}&'
           'language=en&'
           'sortBy=publishedAt&'
           'pageSize=100&'
           f'page={page}&'
           f'apiKey={api_key}')
    response = requests.get(url)
    data = response.json()
    return data


In [5]:
# Obtener todas las noticias (hasta el límite de la API)
todas_las_noticias = []
page = 1
while True:
    data = obtener_noticias(QUERY, FROM_DATE, TO_DATE, NEWS_API_KEY, page=page)
    articles = data.get('articles')
    if not articles:
        break
    todas_las_noticias.extend(articles)
    page += 1
    if page > 5:  # Ajusta este límite según las restricciones de la API
        break

# Crear DataFrame con las noticias
df_noticias = pd.DataFrame(todas_las_noticias)
df_noticias = df_noticias[['publishedAt', 'title', 'description']]
df_noticias['publishedAt'] = pd.to_datetime(df_noticias['publishedAt']).dt.date
df_noticias['content'] = df_noticias['title'].fillna('') + '. ' + df_noticias['description'].fillna('')
df_noticias = df_noticias[['publishedAt', 'content']]
df_noticias


,publishedAt,content
0,2024-12-01,WE ARE THE LAND – UK Farmers Protesting Agains...
1,2024-12-01,55K Bitcoin worth $5.34B pulled from exchanges...
2,2024-12-01,Dogecoin To $3? Expert Says The Countdown Has ...
3,2024-12-01,80/20 Living. An interesting dynamic way to li...
4,2024-12-01,FinTech CEOs Expose How Feds Colluded In ‘Deba...
...,...,...
95,2024-11-29,The Philosophy of Property. I’ve never owned p...
96,2024-11-29,Ripple Secures Major Win Amid Gensler’s Resign...
97,2024-11-29,"Thanksgiving, Economics, and Sound Money. By M..."
98,2024-11-29,8 Bit Essay. To transform this essay into an “...


In [6]:
# Cargar el modelo de análisis de sentimiento
sentimiento = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

# Función para obtener la puntuación de sentimiento
def obtener_puntuacion(texto):
    try:
        resultado = sentimiento(texto[:512])[0]  # Limitar a 512 caracteres
        puntuacion = int(resultado['label'].split()[0])  # Extraer la puntuación (1-5 estrellas)
        return puntuacion
    except:
        return 3  # Neutral en caso de error

# Aplicar el análisis de sentimiento a cada noticia
df_noticias['sentimiento_puntuacion'] = df_noticias['content'].apply(obtener_puntuacion)


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [7]:
# Mapear las puntuaciones numéricas a categorías
def mapear_sentimiento(puntuacion):
    if puntuacion >= 4:
        return 'Positivo'
    elif puntuacion == 3:
        return 'Neutral'
    else:
        return 'Negativo'

df_noticias['sentimiento'] = df_noticias['sentimiento_puntuacion'].apply(mapear_sentimiento)


In [8]:
# Agrupar noticias por fecha
df_agrupado = df_noticias.groupby('publishedAt').agg({
    'content': ' '.join,  # Combinar todas las noticias del día
    'sentimiento_puntuacion': 'mean'  # Obtener la puntuación promedio
}).reset_index()

# Mapear la puntuación promedio a categorías de sentimiento
df_agrupado['sentimiento'] = df_agrupado['sentimiento_puntuacion'].apply(mapear_sentimiento)


In [9]:
# Renombrar columnas y seleccionar las necesarias
df_final = df_agrupado.rename(columns={'publishedAt': 'fecha', 'content': 'noticias'})
df_final = df_final[['fecha', 'noticias', 'sentimiento']]

# Mostrar el DataFrame final
print(df_final)


        fecha                                           noticias sentimiento
0  2024-11-29  Earn up to US$52 in Crypto for Completing Lear...    Negativo
1  2024-11-30  Ethereum’s Surging Price Is Helping Spot Ether...    Negativo
2  2024-12-01  WE ARE THE LAND – UK Farmers Protesting Agains...    Negativo
